# E-commerce A/B Testing: Conversion Rate Optimization

### Load and Clean the data

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("ecommerce_clickstream_transactions_.csv")

# Parse timestamp
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

# Basic cleanup
df["EventType"] = df["EventType"].astype(str).str.lower().str.strip()
df["ProductID"] = df["ProductID"].astype(str)

df.head(), df.shape


(   UserID  SessionID                  Timestamp     EventType  ProductID  \
 0       1          1 2024-07-07 18:00:26.959902     page_view        nan   
 1       1          1 2024-03-05 22:01:00.072000     page_view        nan   
 2       1          1 2024-03-23 22:08:10.568453  product_view  prod_8199   
 3       1          1 2024-03-12 00:32:05.495638   add_to_cart  prod_4112   
 4       1          1 2024-02-25 22:43:01.318876   add_to_cart  prod_3354   
 
    Amount Outcome  
 0     NaN     NaN  
 1     NaN     NaN  
 2     NaN     NaN  
 3     NaN     NaN  
 4     NaN     NaN  ,
 (74817, 7))

### Building a session level dataset

In [2]:
# Flags
df["is_page_view"] = (df["EventType"] == "page_view").astype(int)
df["is_product_view"] = (df["EventType"] == "product_view").astype(int)
df["is_add_to_cart"] = (df["EventType"] == "add_to_cart").astype(int)
df["is_purchase"] = (df["EventType"] == "purchase").astype(int)

# Session-level aggregation
session = (
    df.groupby(["UserID", "SessionID"])
      .agg(
          session_start=("Timestamp", "min"),
          session_end=("Timestamp", "max"),
          page_views=("is_page_view", "sum"),
          product_views=("is_product_view", "sum"),
          add_to_cart=("is_add_to_cart", "max"),   # did it happen at least once?
          purchase=("is_purchase", "max"),         # did it happen at least once?
          revenue=("Amount", "sum"),               # only purchases have Amount
      )
      .reset_index()
)

session["session_duration_sec"] = (session["session_end"] - session["session_start"]).dt.total_seconds()
session["bounce"] = ((session["page_views"] > 0) & (session["product_views"] == 0) & (session["add_to_cart"] == 0) & (session["purchase"] == 0)).astype(int)

session.head()


,UserID,SessionID,session_start,session_end,page_views,product_views,add_to_cart,purchase,revenue,session_duration_sec,bounce
0,1,1,2024-01-01 23:09:51.956825,2024-07-07 18:00:26.959902,3,1,1,0,0.000000,1.622464e+07,0
1,1,2,2024-01-30 21:47:38.829172,2024-06-27 16:17:34.523695,3,1,0,0,0.000000,1.285380e+07,0
2,1,3,2024-01-19 15:04:33.065650,2024-07-17 03:46:13.897763,1,1,1,1,72.913619,1.551130e+07,0
3,1,4,2024-01-02 00:15:51.420238,2024-07-15 16:15:52.074487,2,1,1,1,7.677938,1.690560e+07,0
4,1,5,2024-01-03 23:51:05.729189,2024-06-27 07:40:55.374830,1,1,0,1,998.570616,1.514819e+07,0


### Assigning A/B groups

In [3]:
import hashlib

def assign_variant(user_id: int) -> str:
    h = hashlib.md5(str(user_id).encode()).hexdigest()
    # convert last 8 hex chars to int, mod 2
    return "B" if (int(h[-8:], 16) % 2 == 1) else "A"

session["variant"] = session["UserID"].apply(assign_variant)
session["variant"].value_counts(normalize=True)


variant
B    0.501
A    0.499
Name: proportion, dtype: float64

### Sanity Check: SRM (Sample Ratio Match)

In [4]:
from statsmodels.stats.proportion import proportions_ztest

counts = session["variant"].value_counts()
nobs = counts.sum()
stat, pval = proportions_ztest([counts["A"]], [nobs], value=0.5)

print("A share:", counts["A"]/nobs)
print("SRM p-value:", pval)


A share: 0.499
SRM p-value: [0.84148027]


### Primary analysis: conversion rate lift + significance

In [5]:
summary = session.groupby("variant").agg(
    sessions=("SessionID", "count"),
    conversions=("purchase", "sum"),
    conv_rate=("purchase", "mean"),
    rps=("revenue", "mean"),  # revenue per session
)

summary


,sessions,conversions,conv_rate,rps
variant,,,,
A,4990,3368,0.674950,268.978665
B,5010,3353,0.669261,271.930032


### Running a 2 proportion z-test

In [6]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

A = session[session["variant"]=="A"]
B = session[session["variant"]=="B"]

count = np.array([A["purchase"].sum(), B["purchase"].sum()])
nobs  = np.array([len(A), len(B)])

zstat, pval = proportions_ztest(count, nobs)
lift = (B["purchase"].mean() - A["purchase"].mean()) / A["purchase"].mean()

print("Conversion A:", A["purchase"].mean())
print("Conversion B:", B["purchase"].mean())
print("Relative lift:", lift)
print("p-value:", pval)


Conversion A: 0.6749498997995992
Conversion B: 0.6692614770459082
Relative lift: -0.008427918509773735
p-value: 0.5446067372301173


### Adding confidence intervals

In [7]:
ciA = proportion_confint(count[0], nobs[0], alpha=0.05, method="wilson")
ciB = proportion_confint(count[1], nobs[1], alpha=0.05, method="wilson")
ciA, ciB


((0.6618236527226627, 0.6878069902197589),
 (0.6561083863414833, 0.6821552013545896))

### Secondary metrics (revenue + add-to-cart)

In [8]:
def bootstrap_diff_mean(x, y, iters=5000, seed=42):
    rng = np.random.default_rng(seed)
    diffs = []
    for _ in range(iters):
        xs = rng.choice(x, size=len(x), replace=True)
        ys = rng.choice(y, size=len(y), replace=True)
        diffs.append(np.mean(ys) - np.mean(xs))
    diffs = np.array(diffs)
    return np.mean(diffs), np.quantile(diffs, [0.025, 0.975])

diff, ci = bootstrap_diff_mean(A["revenue"].values, B["revenue"].values)
print("Revenue/session diff (B-A):", diff)
print("95% CI:", ci)


Revenue/session diff (B-A): 2.8317463150002533
95% CI: [-8.41991079 14.10171102]


### Funnel analysis

In [9]:
funnel = session.groupby("variant").agg(
    sessions=("SessionID", "count"),
    pv_rate=("product_views", lambda x: (x>0).mean()),
    atc_rate=("add_to_cart", "mean"),
    purchase_rate=("purchase", "mean"),
)

funnel


,sessions,pv_rate,atc_rate,purchase_rate
variant,,,,
A,4990,0.674749,0.676353,0.674950
B,5010,0.673852,0.677046,0.669261


### Segmentation

In [10]:
# New vs returning proxy
first_session_time = session.groupby("UserID")["session_start"].min()
session = session.join(first_session_time, on="UserID", rsuffix="_first")
session["is_new_user"] = (session["session_start"] == session["session_start_first"]).astype(int)

seg = session.groupby(["variant", "is_new_user"])["purchase"].mean().unstack()
seg


is_new_user,0,1
variant,,
A,0.671565,0.705411
B,0.665558,0.702595


#### Conversion A ≈ 0.675

#### Conversion B ≈ 0.669

#### p-value ≈ 0.545 (not significant)

#### Variant B is not clearly better